In [5]:
import re #regex for password criteria checking
import getpass #password input box
import json
from pathlib import Path
import os.path

In [6]:
###########################################################
# Functions 
# This function takes in an email and password, and returns true/false based on successful authentication
def loginAuthentication(emailAddress, password):
    if(False): #needs to be changed to check for correct info
        print('Incorrect email/password combination\n')
        return False #keeps the loop running until successful combination
    else:
        return True #exits while loop
        print('\n')

# New user generation, should generate a user and interface the file to save
def newUser():
    try_flag = True # used for password match checking

    fullName = input("Enter Full Name: ")
    email = input("Enter Email Address: ")
    while 1: # prompt the user until the passwords match
        if try_flag == False:
            # create junk values, user won't be created
            fullName = ""
            email = ""
            passWd = ""
            break
        passWd = input("Input Password: ")
        passWd_Check = input("Re-Enter Password: ")
    
        if passWd != passWd_Check:
            while 1: # prompt until accepted "try again?" answer
                try_again = input("Passwords do not match, try again? (y/n):")
                if (try_again == 'y' or try_again == 'Y'):
                    try_flag = True
                    break
                elif (try_again == 'n' or try_again == 'N'):
                    try_flag = False
                    break
                else: # invalid answer
                    print("User did not input y/n")
                
        else:
            print("Passwords Match.")
            break
    user = {
        "Name": fullName,
        "Email": email,
        "Password": passWd
    }
    return user # return the user object to main to be saved

In [7]:



# data object that stores the user's 
class UserData:
    userData_file_path = Path(".SFTP_USERS.dat")
    temp_data_file_path = Path(".SFTP_USERS.tmp")
    data = None
    enc_data = None
    data_fp = None
    temp_fp = None
    
    def __init__(self,filePath=userData_file_path,tempFile=temp_data_file_path):
        if os.path.exists(filePath):
            #self.data_fp = open(filePath, "w+")
            #self.temp_fp = open(tempFile, "w+")
            #self.data = json.load(self.data_fp)
        else:
            self.data = dict()
            #self.data_fp = open(filePath, "w+")
            #self.temp_fp = open(tempFile, "w+")
    
    def __del__(self):
        # encrypt data
        # write encrypted data to temp file
        # overwrite data file with temp file
        # close file pointers
        print("UserData object __del__ method called")
        return None
    
    def addUser(self, user):
        self.data[user['Email']] = user
        self.data[user['Email']]['Contacts'] = {}
    
    def isUserRegistered(self, user):
        if user['Email'] in self.data.keys():
            return True
        else:
            return False
    
    def getContacts(self, user):
        if self.isUserRegistered(user):
            return self.data[user['Email']]['Contacts']
        else:
            return []
    
    def doesContactExist(self, user, contact):
        if self.isUserRegistered(user) and contact in self.data[user['Email']]['Contacts']:
            return True
        else:
            return False
    
    def addContact(self, user, contact):
        self.data[user['Email']]['Contacts'].append(contact)
        # TODO: add contact info to data
        # encrypt data
        #writeData(self)
        return None
    
    def removeContact(self, user, contact):
        if self.doesContactExist(user, contact):
            self.data[user['Email']]['Contacts'].remove(contact)
            return True
        else:
            return False
    
    def loadData(self):
        # TODO:
        return None
    
    def writeData(self):
        # TODO:
        return None







IndentationError: expected an indented block (<ipython-input-7-84c58826c637>, line 15)

In [8]:
###########################################################
# Start of main program
#usersObj = UserData()
userList = False # If there are no users in file 
#samuel insert code around here for file checking

#If there are no users in file, create initial user in while loop
while (userList == False):
    print('No users are registered with this client')
    firstUser= input('Would you like to register a new user now? (y/n): ')
    if (firstUser == 'Y' or firstUser == 'y'):
        
        # create the new user
        user = newUser()
        if (user["Name"] == "" or user["Email"] == "" or user["Password"] == ""): # was a contact unable to be created
            print('You cannot use this system without a user registered')
        #else: # user created, now store it         
             
        
        userList = True #can be changed with user registration code (must not == False to exit loop)
    else:
        print('You cannot use this system without a user registered')

print('\n$Welcome to Secure Drop!$ \n--- Login ---')

loginSuccess = False # Exits the while loop when not False
while (loginSuccess == False):
    emailAddress = input('Email Address:')
    userPassword = getpass.getpass('Password:')
    loginSuccess = loginAuthentication(emailAddress, userPassword) # Changes the boolean to True if successful
print("Login was successful!")

exit_shell = False # Exits the while loop when not False
while (exit_shell == False):
    # 'shell' prefix for each line
    textInput = input('$secure_drop>')
    
    # Add command
    if(textInput == 'add'):
        # Adds users to the contact list for the logged in user
        print('\n--- Add Contact ---')
        input('Enter Full Name: ')
        input('Enter Email Address: ')
        
        if(True): #needs to check our implementation of user storage
            print('Contacted added to List')
        else:
            print('Failed to add contact')
            
    # List command
    elif(textInput == 'list'):
        #molly/sammy code to pull the list of users from the file
        print('test')
        
        
    # Send command
    elif(textInput == 'send'):
        print('test2')
        
    
    # Help command
    elif(textInput == 'help'):
        print('\t"add" -> Add a new contact \n\t\"list\" -> List all online contacts \n\t\"send\" -> Transfer file to contact \n\t\"exit\" -> Exit SecureDrop')
        
    # Exit command
    elif(textInput == 'exit'):
        exit_shell= True
        
print('Exiting Secure Drop')

No users are registered with this client
Would you like to register a new user now? (y/n): y
Enter Full Name: kyle_jolly
Enter Email Address: kyle_jolly@uml.eco
Input Password: Kykyky-1
Re-Enter Password: Kykykyk-2
Passwords do not match, try again? (y/n):y


NameError: name 'true' is not defined

In [18]:
usersObj.data

{}